# TinyAES Attack

AES encryption :

/home/gata/Bureau/MA3/semester_project/project/hardware/victims/firmware/crypto/tiny-AES128-C

---
Objective : Attack protected AES

---

## AES Python model

In [ ]:
import numpy as np

sbox = [
    # 0    1    2    3    4    5    6    7    8    9    a    b    c    d    e    f 
    0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5,0x30,0x01,0x67,0x2b,0xfe,0xd7,0xab,0x76, # 0
    0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0,0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0, # 1
    0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc,0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15, # 2
    0x04,0xc7,0x23,0xc3,0x18,0x96,0x05,0x9a,0x07,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75, # 3
    0x09,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0,0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84, # 4
    0x53,0xd1,0x00,0xed,0x20,0xfc,0xb1,0x5b,0x6a,0xcb,0xbe,0x39,0x4a,0x4c,0x58,0xcf, # 5
    0xd0,0xef,0xaa,0xfb,0x43,0x4d,0x33,0x85,0x45,0xf9,0x02,0x7f,0x50,0x3c,0x9f,0xa8, # 6
    0x51,0xa3,0x40,0x8f,0x92,0x9d,0x38,0xf5,0xbc,0xb6,0xda,0x21,0x10,0xff,0xf3,0xd2, # 7
    0xcd,0x0c,0x13,0xec,0x5f,0x97,0x44,0x17,0xc4,0xa7,0x7e,0x3d,0x64,0x5d,0x19,0x73, # 8
    0x60,0x81,0x4f,0xdc,0x22,0x2a,0x90,0x88,0x46,0xee,0xb8,0x14,0xde,0x5e,0x0b,0xdb, # 9
    0xe0,0x32,0x3a,0x0a,0x49,0x06,0x24,0x5c,0xc2,0xd3,0xac,0x62,0x91,0x95,0xe4,0x79, # a
    0xe7,0xc8,0x37,0x6d,0x8d,0xd5,0x4e,0xa9,0x6c,0x56,0xf4,0xea,0x65,0x7a,0xae,0x08, # b
    0xba,0x78,0x25,0x2e,0x1c,0xa6,0xb4,0xc6,0xe8,0xdd,0x74,0x1f,0x4b,0xbd,0x8b,0x8a, # c
    0x70,0x3e,0xb5,0x66,0x48,0x03,0xf6,0x0e,0x61,0x35,0x57,0xb9,0x86,0xc1,0x1d,0x9e, # d
    0xe1,0xf8,0x98,0x11,0x69,0xd9,0x8e,0x94,0x9b,0x1e,0x87,0xe9,0xce,0x55,0x28,0xdf, # e
    0x8c,0xa1,0x89,0x0d,0xbf,0xe6,0x42,0x68,0x41,0x99,0x2d,0x0f,0xb0,0x54,0xbb,0x16  # f
]

def aes_internal(inputdata, key):
    return sbox[inputdata ^ key]

def calc_hamming_weight(n):
    return bin(n).count("1")

HW = []
for i in range(256):
    HW += [calc_hamming_weight(i)]
    
assert HW[aes_internal(0xA1, 0x79)] == 3
assert HW[aes_internal(0x22, 0xB1)] == 5
print("✔️ OK to continue!")

In [ ]:
# See https://github.com/boppreh/aes/

def pad(plaintext):
    padding_len = 16 - (len(plaintext) % 16)
    padding = bytes([padding_len] * padding_len)
    return plaintext + padding

def unpad(plaintext):
    padding_len = plaintext[-1]
    assert padding_len > 0
    message, padding = plaintext[:-padding_len], plaintext[-padding_len:]
    assert all(p == padding_len for p in padding)
    return message

def bytes2matrix(text):
    """ Converts a 16-byte array into a 4x4 matrix.  """
    return [list(text[i:i+4]) for i in range(0, len(text), 4)]

def matrix2bytes(matrix):
    """ Converts a 4x4 matrix into a 16-byte array.  """
    return bytes(sum(matrix, []))

def xor_bytes(a, b):
    """ Returns a new byte array with the elements xor'ed. """
    return bytes(i^j for i, j in zip(a, b))

def print_byte_array_in_hex(array):
    if type(array) == list:
        tmp = []
        for i in range(len(array)):
            tmp += [list(map(lambda elem: hex(elem), array[i]))]
        #print(tmp)
        return tmp
    
    tmp = np.array(list(map(lambda elem: hex(elem), array)))
    #print(tmp)
    return tmp

def print_byte_array_in_int(array):
    if type(array) == list:
        tmp = []
        for i in range(len(array)):
            tmp += [list(map(lambda elem: hex(elem), array[i]))]
        #print(tmp)
        return tmp
    tmp = np.array(list(map(lambda elem: int(elem), array)))
    return tmp

def state_to_hamming_weight(s):
    for i in range(4):
        for j in range(4):
            s[i][j] = HW[s[i][j]]
            
def print_state_to_hex_str(s):
    for i in range(4):
        for j in range(4):
            s[i][j] = hex(s[i][j])
    #print(s)
    state_from_hex_str_to_int(s)
            
def state_from_hex_str_to_int(s):
    for i in range(4):
        for j in range(4):
            s[i][j] = int(s[i][j], 16)

In [ ]:
bytes_msg = b"AES message"
tmp = (matrix2bytes(bytes2matrix(pad(bytes_msg))))
assert unpad(matrix2bytes(bytes2matrix(pad(bytes_msg)))) == bytes_msg
print("✔️ OK to continue!")

In [ ]:
# Addition key or Round key : AK
def add_round_key(s, k):
    for i in range(4):
        for j in range(4):
            s[i][j] ^= k[i][j]

# Sub-bytes or S-Box : SB
def sub_bytes(s):
    if type(s) != list:
        return sbox[s]
    elif type(s[0]) == list:
        for i in range(4):
            for j in range(4):
                s[i][j] = sbox[s[i][j]]
        return s
    elif type(s[0]) == int:
        for i in range(4):
            s[i] = sbox[s[i]]
        return s

# Shift rows or Permutation: PB
def shift_rows(s):
    s[0][1], s[1][1], s[2][1], s[3][1] = s[1][1], s[2][1], s[3][1], s[0][1]
    s[0][2], s[1][2], s[2][2], s[3][2] = s[2][2], s[3][2], s[0][2], s[1][2]
    s[0][3], s[1][3], s[2][3], s[3][3] = s[3][3], s[0][3], s[1][3], s[2][3]
    
def custom_shift_rows(plaintext, index):
    shift_rows = [0x00, 0x01, 0x02, 0x03, 0x05, 0x06, 0x07, 0x04, 0x0A, 0x0B, 0x08, 0x09, 0x0F, 0x0C, 0x0D, 0x0E]
    return s[shift_rows[index]]

# Diffusion layer or Mix Column : MC
xtime = lambda a: (((a << 1) ^ 0x1B) & 0xFF) if (a & 0x80) else (a << 1)
    
def mix_single_column(a):
    t = a[0] ^ a[1] ^ a[2] ^ a[3]
    u = a[0]
    a[0] ^= t ^ xtime(a[0] ^ a[1])
    a[1] ^= t ^ xtime(a[1] ^ a[2])
    a[2] ^= t ^ xtime(a[2] ^ a[3])
    a[3] ^= t ^ xtime(a[3] ^ u)

def mix_columns(s):
    for i in range(4):
        mix_single_column(s[i])
        
# Key scheduling KS
r_con = (
    0x00, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40,
    0x80, 0x1B, 0x36, 0x6C, 0xD8, 0xAB, 0x4D, 0x9A,
    0x2F, 0x5E, 0xBC, 0x63, 0xC6, 0x97, 0x35, 0x6A,
    0xD4, 0xB3, 0x7D, 0xFA, 0xEF, 0xC5, 0x91, 0x39,
)

def expand_key(master_key):
    # Initialize round keys with raw key material.
    key_columns = [bytes(elem) for elem in bytes2matrix(master_key)]
    iteration_size = len(master_key) // 4

    i = 1
    while len(key_columns) < (10 + 1) * 4:
        # Copy previous word.
        word = list(key_columns[-1])

        # Perform schedule_core once every "row".
        if len(key_columns) % iteration_size == 0:
            # Circular shift.
            word.append(word.pop(0))
            # Map to S-BOX.
            word = [sub_bytes(b) for b in word]
            # XOR with first byte of R-CON, since the others bytes of R-CON are 0.
            word[0] ^= r_con[i]
            i += 1
        elif len(master_key) == 32 and len(key_columns) % iteration_size == 4:
            # Run word through S-box in the fourth iteration when using a
            # 256-bit key.
            word = [sub_bytes(b) for b in word]

        # XOR with equivalent word from previous iteration.
        word = xor_bytes(word, key_columns[-iteration_size])
        key_columns.append(word)

    # Group key words in 4x4 byte matrices.
    return [key_columns[4*i : 4*(i+1)] for i in range(len(key_columns) // 4)]

In [ ]:
# AES* - 10 rounds
def AES_128_enc(plaintext, key):
    assert len(plaintext) == 16

    plain_state = bytes2matrix(plaintext)

    add_round_key(plain_state, key[0])
    
    for i in range(1, 10):
        sub_bytes(plain_state)
        shift_rows(plain_state)
        mix_columns(plain_state)
        add_round_key(plain_state, key[i])

    sub_bytes(plain_state)
    shift_rows(plain_state)
    add_round_key(plain_state, key[10])

    return matrix2bytes(plain_state)

Test AES python implementation :

In [ ]:
plaintext = bytes([0x00 for i in range(16)])
key = expand_key([0x00 for i in range(16)])

assert AES_128_enc(plaintext, key).hex() == "66e94bd4ef8a2c3b884cfa59ca342b2e"
print("✔️ OK to continue!")

## AES with ChipWhisperer board

In [ ]:
def encrypt_p_power(p):
    scope.arm()
    
    target.simpleserial_write('p', bytearray(p))
    
    time.sleep(0.012)
    ret = scope.capture()
    
    trace_array = scope.get_last_trace()
    return trace_array

# Encrypt plaintext p n-times and returns average power trace
def encrypt_p_power_repeat_n(p, n):
    res = np.array(encrypt_p_power(p))
    for i in range(n - 1):
        tmp = np.array(encrypt_p_power(p))
        res += tmp
    return (1/n*res).tolist()

def min_moving_average(power_trace, list_, half_widow_size = 1):
    res = 10
    for i in list_:
        tmp = power_trace[i-half_widow_size:i+half_widow_size].mean(0)
        res = min(tmp, res)
    return res

# Encrypt plaintext p n-times and returns average of minimum power trace of indices
def encrypt_p_power_repeat_n_min(p, n, list_):
    #res = np.min(np.array(encrypt_p_power(p))[list_])
    res = min_moving_average(encrypt_p_power(p), list_)
    for i in range(n - 1):
        #tmp = np.min(np.array(encrypt_p_power(p))[list_])
        #res = np.miniumum(res, tmp)
        tmp = min_moving_average(encrypt_p_power(p), list_)
        res += tmp
    return 1/n*res

def sort_moving_average(power_trace, list_, half_widow_size = 1):
    res = []
    for i in list_:
        tmp = power_trace[i-half_widow_size:i+half_widow_size].mean(0)
        res += [tmp]
    return np.sort(np.array(res))

# Encrypt plaintext p n-times and returns average of sort power traces
def encrypt_p_power_repeat_n_sort(p, n, list_, half_widow_size=1):
    res = sort_moving_average(encrypt_p_power(p), list_, half_widow_size)
    for i in range(n - 1):
        tmp = sort_moving_average(encrypt_p_power(p), list_, half_widow_size)
        res += tmp
    return 1/n*res

def sort_moving_average_abs(power_trace, list_, half_widow_size = 13):
    res = []
    for i in list_:
        tmp = np.abs(power_trace[i-half_widow_size:i+half_widow_size]).mean(0)
        res += [tmp]
    return np.sort(np.array(res))

# Encrypt plaintext p n-times and returns average of sort power traces
def encrypt_p_power_repeat_n_sort_abs(p, n, list_, half_widow_size = 13):
    res = sort_moving_average_abs(encrypt_p_power(p), list_, half_widow_size)
    for i in range(n - 1):
        tmp = sort_moving_average_abs(encrypt_p_power(p), list_, half_widow_size)
        res += tmp
    return 1/n*res

def moving_average_abs(power_trace, list_, half_widow_size = 3):
    res = []
    for i in list_:
        tmp = np.abs(power_trace[i-half_widow_size:i+half_widow_size]).mean(0)
        res += [tmp]
    return np.array(res)

# Encrypt plaintext p n-times and returns average of power traces
def encrypt_p_power_repeat_n_abs(p, n, list_, half_widow_size = 3):
    res = moving_average_abs(encrypt_p_power(p), list_, half_widow_size)
    for i in range(n - 1):
        tmp = moving_average_abs(encrypt_p_power(p), list_, half_widow_size)
        res += tmp
    return 1/n*res

# Encrypt plaintext p n-times and returns average bounds of maximum sort power traces
def encrypt_p_power_repeat_n_sort_bounds(p, n, list_, half_widow_size = 5):
    res_low_bound = sort_local_max(encrypt_p_power(p), list_, half_widow_size)
    res_high_bound = res_low_bound.copy()
    for i in range(n - 1):
        tmp = sort_local_max(encrypt_p_power(p), list_, half_widow_size)
        res_low_bound = np.minimum(res_low_bound, tmp)
        res_high_bound = np.maximum(res_high_bound, tmp)
    return res_low_bound, res_high_bound

def sort_local_max(power_trace, list_, half_widow_size = 3):
    res = []
    for i in list_:
        tmp = np.max(power_trace[i-half_widow_size:i+half_widow_size])
        res += [tmp]
    return np.sort(np.array(res))

def encrypt_p_power_repeat_n_sort_max(p, n, list_, half_widow_size=3):
    res = sort_local_max(encrypt_p_power(p), list_, half_widow_size)
    for i in range(n - 1):
        tmp = sort_local_max(encrypt_p_power(p), list_, half_widow_size)
        res += tmp
    return 1/n*res

def moving_average_abs_bounds(power_trace, list_bounds):
    res = []
    for i in list_bounds:
        tmp = np.abs(power_trace[i[0]:i[1]]).mean(0)
        res += [tmp]
    return np.array(res)

def encrypt_p_power_repeat_n_abs_bounds(p, n, list_bounds):
    res = moving_average_abs_bounds(encrypt_p_power(p), list_bounds)
    for i in range(n - 1):
        tmp = moving_average_abs_bounds(encrypt_p_power(p), list_bounds)
        res += tmp
    return 1/n*res

# with no average
def moving_average_abs_bounds_no_average(power_trace, list_bounds):
    res = []
    for i in list_bounds:
        tmp = np.abs(power_trace[i[0]:i[1]])
        res += [tmp]
    return np.array(res)

def encrypt_p_power_repeat_n_abs_bounds_no_average(p, n, list_bounds):
    res = moving_average_abs_bounds_no_average(encrypt_p_power(p), list_bounds)
    for i in range(n - 1):
        tmp = moving_average_abs_bounds_no_average(encrypt_p_power(p), list_bounds)
        res += tmp
    return 1/n*res

In [ ]:
def set_up(key):
    target.set_key(bytearray(key))

In [ ]:
def find_active_column(avg_trace_shift, avg_trace):
    # argmax = np.argmax(np.abs(np.array(avg_trace_shift[MC_low_index:MC_high_index]) - np.array(avg_trace[MC_low_index:MC_high_index])))
    argmax = np.argmax(np.abs(np.array(avg_trace_shift[:]) - np.array(avg_trace[:])))
    #print(f"argmax : {argmax}")
    if argmax >= 0 and argmax <= 220:
        return 0
    if argmax <= 370:
        return 1
    if argmax <= 550:
        return 2
    if argmax <= 720:
        return 3
    
set_u = 0xFF

def find_u_activating_jth_column(avg_trace, plaintext, j):
    u_result = []
    for i in range(16):
        assert plaintext[i] != set_u
        tmp = plaintext[i]
        plaintext[i] = set_u
        avg_trace_shift = encrypt_p_power_repeat_n(plaintext, 100)
        #print(f"j : {j}")
        #print(f"i : {i}")
        if find_active_column(avg_trace_shift, avg_trace) == j:
            u_result += [i]
        plaintext[i] = tmp
    return u_result

In [ ]:
def find_best_occurances(sub_trace, overall_trace):
    length = np.shape(sub_trace)[0]
    best_i = 0
    avg = float('inf')
    for i in range(np.shape(overall_trace)[0] - length):
        avg_tmp = np.sum(np.abs(overall_trace[i:i+length] - sub_trace), axis = 0)
        if avg_tmp < avg:
            best_i = i 
            avg = avg_tmp
    return best_i

def insert_0(index_0_left, index_0_right, overall_trace):
    assert index_0_right >= index_0_left
    assert index_0_left >= 0
    return np.concatenate((overall_trace[:index_0_left], np.zeros(index_0_right - index_0_left - 1), overall_trace[index_0_left:]))

## Random permutation add round key

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM='CW308_STM32F4'
CRYPTO_TARGET='TINYAES128C'
%run "Setup_Scripts/Setup_Generic.ipynb"
AES_TINY_VERSION='V4'

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$AES_TINY_VERSION"
cd hardware/victims/firmware/simpleserial-aes-5
make PLATFORM=$1 CRYPTO_TARGET=$2  AES_TINY_VERSION=$3

In [ ]:
cw.program_target(scope, prog, "hardware/victims/firmware/simpleserial-aes-5/simpleserial-aes-{}.hex".format(PLATFORM))
project = cw.create_project("traces/STM32F4_HW_AES_5.cwp", overwrite=True)

In [ ]:
plaintext = [0x00 for i in range(16)]
key = [0x2b, 0x7e, 0x15, 0x16, 0x28, 0xae, 0xd2, 0xa6, 0xab, 0xf7, 0x15, 0x88, 0x09, 0xcf, 0x4f, 0x3c]
set_up(key)

key = expand_key(key)
key = key[0]
print_byte_array_in_hex(key)

MAX_cwlite = 24400
scope.adc.samples = 20000
scope.adc.offset = 0

avg_trace = encrypt_p_power_repeat_n(plaintext, 100)

In [ ]:
#(np.argwhere(np.abs(np.array(avg_trace_2[0:1000]) - np.array(avg_trace[0:1000])) > 1.0e-02)[0, 0])

In [ ]:
%matplotlib notebook
import matplotlib.pylab as plt

plt.rcParams["figure.figsize"] = (8,3)

plt.figure()
plt.plot(avg_trace)
#plt.plot(insert_0(450, 1292, avg_trace_2), "r")
plt.show()

BlockCopy: 20 - 500

Randomize key 0 : 500 - 4345

Round key (AK) : 4345 - 5217

Sub bytes (SB) : 5217 - 6127

Shift rows (PB) : 6127 - 8873

Mix Columns (MC) : 8873 - 9654

Randomize key 1 : 9654 - 13695

Round key (AK) : 13450 - 14292


In [ ]:
spikes = np.array(avg_trace[:5217])
index = np.where(spikes >= 0.065)[0]
index = index[index > 4345]
index = index - 4345

In [ ]:
assert len(index) == 16

In [ ]:
plaintext = [0x01 for i in range(16)]
key = [0x2b, 0x7e, 0x15, 0x16, 0x28, 0xae, 0xd2, 0xa6, 0xab, 0xf7, 0x15, 0x88, 0x09, 0xcf, 0x4f, 0x3c]
set_up(key)

key = expand_key(key)
key = key[0]
print(index)
print_byte_array_in_hex(key)

scope.adc.samples = 5217 - 4345
scope.adc.offset = 4345

#encrypt_p_power_repeat_n(plaintext, 1)
#key = target.simpleserial_read('r', 16)
#key = print_byte_array_in_hex(key)


L = []
for t in range(256):
    #plaintext[0] = t
    plaintext = [t for i in range(16)]
    e = encrypt_p_power_repeat_n_min(plaintext, 500, index)
    #e = e[index[0]]
    #e = e.mean(axis = 0)
    
    print(f"For p = [{hex(t)}*16], e_count = {e}")
    L += [e]

L = np.array(L)
idx = np.argsort(L)[:32]
for i in list(idx):
    print(f"{hex(i)} ", end="")


## Shuffle add round key

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM='CW308_STM32F4'
CRYPTO_TARGET='TINYAES128C'
%run "Setup_Scripts/Setup_Generic.ipynb"
AES_TINY_VERSION='V5'

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$AES_TINY_VERSION"
cd hardware/victims/firmware/simpleserial-aes-6
make PLATFORM=$1 CRYPTO_TARGET=$2  AES_TINY_VERSION=$3

In [ ]:
cw.program_target(scope, prog, "hardware/victims/firmware/simpleserial-aes-6/simpleserial-aes-{}.hex".format(PLATFORM))
project = cw.create_project("traces/STM32F4_HW_AES_6.cwp", overwrite=True)

In [ ]:
plaintext = [0xFF for i in range(16)]
key = [0x2b, 0x7e, 0x15, 0x16, 0x28, 0xae, 0xd2, 0xa6, 0xab, 0xf7, 0x15, 0x88, 0x09, 0xcf, 0x4f, 0x3c]
# key = [0xFF] + [0x00 for i in range(15)]

set_up(key)

key = expand_key(key)
key = key[0]
print_byte_array_in_hex(key)

MAX_cwlite = 24400
scope.adc.samples = 10000
scope.adc.offset = 0

avg_trace = encrypt_p_power_repeat_n(plaintext, 500)

In [ ]:
%matplotlib notebook
import matplotlib.pylab as plt

plt.rcParams["figure.figsize"] = (8,3)

plt.figure()
plt.plot(avg_trace)
#plt.plot(insert_0(3860, 5413, avg_trace_2), "r")
#plt.plot(avg_trace_2)
plt.show()

In [ ]:
# np.savetxt('output.dat', np.array([np.array(range(1553)), np.array(avg_trace[3860:5413])]).T, fmt='%d %f')

BlockCopy: 20 - 500

Randomize key 0 : 500 - 3860

Round key (AK) : 3860 - 5413

Sub bytes (SB) : 5413 - 6138

Shift rows (PB) : 6138 - 8885

Mix Columns (MC) : 8885 - 9634

Randomize key 1 : 9634 - 

Round key (AK) :  - 

In [ ]:
spikes = np.array(avg_trace[:5413])
index = np.where(spikes >= 0.0620)[0]
index = index[index > 3860]
index = index - 3860

In [ ]:
index = [64, 156, 248, 340, 432, 524, 616, 708, 800, 892, 984, 1076, 1168, 1260, 1352, 1444]

In [ ]:
assert len(index) == 16

In [ ]:
plaintext = [0x01 for i in range(16)]
key = [0x2b, 0x7e, 0x15, 0x16, 0x28, 0xae, 0xd2, 0xa6, 0xab, 0xf7, 0x15, 0x88, 0x09, 0xcf, 0x4f, 0x3c]
set_up(key)

key = expand_key(key)
key = key[0]

scope.adc.samples = 5413 - 3860
scope.adc.offset = 3860

L = []
for t in range(256):
    #plaintext[0] = t
    plaintext = [t for i in range(16)]
    e = encrypt_p_power_repeat_n_min(plaintext, 500, index)
    #e = e[index[0]]
    #e = e.mean(axis = 0)
    
    print(f"For p = [{hex(t)}*16], e_count = {e}")
    L += [e]

L = np.array(L)
idx = np.argsort(L)[0:16]
for i in list(idx):
    print(f"{hex(i)} ", end="")


## Algo2 : compute bytes hamming weight after AK

In [ ]:
plaintext = [0x00 for i in range(16)]
key = [0x00, 0x01, 0x03, 0x30, 0x70, 0x07, 0xF0, 0xC3, 0x3C, 0x0F, 0x1F, 0xF1, 0x3F, 0xF3, 0x7F, 0xFF]
set_up(key)
key = expand_key(key)
index = [64, 156, 248, 340, 432, 524, 616, 708, 800, 892, 984, 1076, 1168, 1260, 1352, 1444]

scope.adc.samples = 5413 - 3860
scope.adc.offset = 3860

t=0
plaintext_state = bytes2matrix(plaintext)
add_round_key(plaintext_state, key[0])
state_to_hamming_weight(plaintext_state)
res = matrix2bytes(plaintext_state)
res = np.array(print_byte_array_in_int(res))
print(f"For p = [{hex(t)}*16], Order HW[p_AK] = {np.sort(res)}")

e = encrypt_p_power_repeat_n_sort(plaintext, 5000, index)
print(f"For p = [{hex(t)}*16], e_counts = {e}")
e = np.array(e)


In [ ]:
profile = []
for i in range(9):
    profile += [e[np.where(np.sort(res) == i)[0]].mean(0)]
print(profile)

In [ ]:
# import random 

key = [0x2b, 0x7e, 0x15, 0x16, 0x28, 0xae, 0xd2, 0xa6, 0xab, 0xf7, 0x15, 0x88, 0x09, 0xcf, 0x4f, 0x3c]
set_up(key)
index = [64, 156, 248, 340, 432, 524, 616, 708, 800, 892, 984, 1076, 1168, 1260, 1352, 1444]
key = expand_key(key)

scope.adc.samples = 5413 - 3860
scope.adc.offset = 3860

profile = [0x00 for i in range(9)]
profile_count = [0x00 for i in range(9)]
list_ = [[] for i in range(9)]

for t in range(10):
    #t = random.randrange(0, 256)
    print(str(t).zfill(3), end = "\r")
    plaintext = [t for i in range(16)]

    plaintext_state = bytes2matrix(plaintext)
    add_round_key(plaintext_state, key[0])
    state_to_hamming_weight(plaintext_state)
    res = matrix2bytes(plaintext_state)
    res = np.sort(np.array(print_byte_array_in_int(res)))

    e = encrypt_p_power_repeat_n_sort(plaintext, 500, index)
    
    for i in range(9):
        tmp = e[np.where(res == i)[0]]
        if len(tmp) > 0:
            profile[i] += tmp.mean(0)
            profile_count[i] += 1
            
            list_[i] += list(tmp)
            
        
profile = np.array(profile)/np.array(profile_count)
print(profile)



In [ ]:
[elem.sort() for elem in list_]

print(list_)

In [ ]:
def find_HW_from_power_value_profile(val, profile):
    return np.abs(profile - val).argmin()

def find_HWs_from_power_trace_profile(val_list, profile):
    return [find_HW_from_power_value_profile(elem, profile) for elem in list(val_list)]

In [ ]:
plaintext = [0x01 for i in range(16)]
key = [0x2b, 0x7e, 0x15, 0x16, 0x28, 0xae, 0xd2, 0xa6, 0xab, 0xf7, 0x15, 0x88, 0x09, 0xcf, 0x4f, 0x3c]
set_up(key)
key = expand_key(key)
index = [64, 156, 248, 340, 432, 524, 616, 708, 800, 892, 984, 1076, 1168, 1260, 1352, 1444]

scope.adc.samples = 5413 - 3860
scope.adc.offset = 3860

L = []
for t in range(10):
    plaintext = [t for i in range(16)]
    
    plaintext_state = bytes2matrix(plaintext)
    add_round_key(plaintext_state, key[0])
    state_to_hamming_weight(plaintext_state)
    res = matrix2bytes(plaintext_state)
    res = print_byte_array_in_int(res)
    print(f"For p = [{hex(t)}*16], ordered HW[p_AK] = {np.sort(res)}")

    e = encrypt_p_power_repeat_n_sort(plaintext, 500, index)
    # print(f"For p = [{hex(t)}*16], e_counts = {e}")
    print(f"        estimated ordered HW[p_AK] = {np.array(find_HWs_from_power_trace_profile(e, profile))}")
    
    L += [e]

## Algo3 : Excet key permutation recovery

In [ ]:
key = [0x2b, 0x7e, 0x15, 0x16, 0x28, 0xae, 0xd2, 0xa6, 0xab, 0xf7, 0x15, 0x88, 0x09, 0xcf, 0x4f, 0x3c]

set_up(key)

scope.adc.samples = 5413 - 3860
scope.adc.offset = 3860
key_recovery = []

plaintext = [0x00 for i in range(16)]
assert 0x00 not in key

profile = [0.01615625, 0.02376363, 0.0316329, 0.04111055, 0.0494502, 0.05807837, 0.06740578, 0.07636849]


for i in range(16):
    print(f"Recover {i}th key byte")
    for j in range(16):
        plaintext[i] = key[j]

        e = encrypt_p_power_repeat_n_sort(plaintext, 500, index)
        if min(find_HWs_from_power_trace_profile(e, profile)) == 0:
            print(f"k_{i} = {hex(key[j])}")
            key_recovery += [key[j]]

print(key_recovery)

## Substitution recovery (protected SB via shuffle instruction)

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM='CW308_STM32F4'
CRYPTO_TARGET='TINYAES128C'
%run "Setup_Scripts/Setup_Generic.ipynb"
AES_TINY_VERSION='V6'

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$AES_TINY_VERSION"
cd hardware/victims/firmware/simpleserial-aes-7
make PLATFORM=$1 CRYPTO_TARGET=$2  AES_TINY_VERSION=$3

In [ ]:
cw.program_target(scope, prog, "hardware/victims/firmware/simpleserial-aes-7/simpleserial-aes-{}.hex".format(PLATFORM))
project = cw.create_project("traces/STM32F4_HW_AES_7.cwp", overwrite=True)


In [ ]:
plaintext = [0x00 for i in range(16)]
key = [0x2b, 0x7e, 0x15, 0x16, 0x28, 0xae, 0xd2, 0xa6, 0xab, 0xf7, 0x15, 0x88, 0x09, 0xcf, 0x4f, 0x3c]
set_up(key)

key = expand_key(key)
key = key[0]
print_byte_array_in_hex(key)

MAX_cwlite = 24400
scope.adc.samples = 10000
scope.adc.offset = 0

avg_trace = encrypt_p_power_repeat_n(plaintext, 500)
# avg_trace_2 = encrypt_p_power_repeat_n(plaintext, 500)


In [ ]:
%matplotlib notebook
import matplotlib.pylab as plt

plt.rcParams["figure.figsize"] = (8,3)

plt.figure()
plt.plot(avg_trace[8523:9831])
# plt.plot(insert_0(8523, 9831, avg_trace_2), "r")
plt.show()

BlockCopy: 20 - 500

Randomize key 0 : 500 - 3860

Round key (AK) : 3860 - 5413

Randomize sbox instr 0 : 5413 - 8523

Sub bytes (SB) : 8523 - 9831

Shift rows (PB) : 9831 - 

Mix Columns (MC) :  - 

Randomize key 1 :  - 

Round key (AK) :  - 

In [ ]:
spikes = np.array(avg_trace[:9831])
index = np.where(spikes >= 0.062)[0]
#index = np.where(spikes <= -0.075)[0]
index = index[index > 8523]
index = index - 8523

In [ ]:
print(index)

In [ ]:
# Negative spikes
index_1 = [66, 142, 218, 294, 370, 446, 522, 598, 674, 750, 826, 902, 978, 1054, 1130, 1198]
print(len(index_1))


In [ ]:
index_2 = [73, 149, 225, 301, 377, 453, 529, 605, 681, 757, 833, 909, 985, 1061, 1137, 1205]
assert len(index_2) == 16


In [ ]:
index_3 = [65, 141, 217, 293, 369, 445, 521, 597, 673, 749, 825, 901, 977, 1053, 1129, 1205]

#index_bounds = np.where(np.abs(np.array(e[2]) - np.array(e[0])) >= 0.01)[0]
index_bounds = [[58, 78], [134, 155], [210, 231], [286, 306], [362, 382], [438, 459], [514, 535], [590, 611], [666, 687], [742, 763], [818, 839], [894, 915], [970, 991], [1046, 1067], [1122, 1143], [1198, 1231]]


## Algo4 : compute bytes hamming weight after SB

In [ ]:
plaintext = [0x00 for _ in range(16)]
key = [0x2b, 0x7e, 0x15, 0x16, 0x28, 0xae, 0xd2, 0xa6, 0xab, 0xf7, 0x15, 0x88, 0x09, 0xcf, 0x4f, 0x3c]
set_up(key)
key = expand_key(key)
index_2 = [73, 149, 225, 301, 377, 453, 529, 605, 681, 757, 833, 909, 985, 1061, 1137, 1205]

scope.adc.samples = 9831 - 8523
scope.adc.offset = 8523

key = print_byte_array_in_int(key[0])

e = []

#for t in [0x52, 0x17, 0x7d]:
for t in [0x52, 0x09, 0x6a, 0x30, 0xbf, 0x7c, 0x54, 0x72, 0x3a]:
#for t in range(256):
    print(str(t).zfill(3), end = "\r")
    
    for i in range(16):
        plaintext[i] = (int(key[i//4][i%4], 16) ^ t) & 0xFF
        
    #e += [encrypt_p_power_repeat_n(plaintext, 1000)]
    #e += [encrypt_p_power_repeat_n_abs(plaintext, 1000, index_2, 3)] 
    #e += [encrypt_p_power_repeat_n_abs_bounds(plaintext, 1000, index_bounds)]
    # TODO : no average, thus obtain not a number in profiling but trace
    e += [encrypt_p_power_repeat_n_abs_bounds_no_average(plaintext, 1000, index_bounds)]
    
    
e = np.array(e)

In [ ]:
import matplotlib.colors as colors

def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

In [ ]:
%matplotlib notebook
import matplotlib.pylab as plt
import matplotlib as mpl

cmap = mpl.colormaps['hsv']
print(cmap)
x = np.linspace(0.0, 0.8, 256)

plt.rcParams["figure.figsize"] = (8,4)

plt.figure()

for i in range(0, 256):
    plt.plot(e[i][0], color=cmap(x[i]), alpha=1, label=f"{i} : Estimated HW_h(SB) = {HW[i]}")
    #print(sort_pw_trace_16_spikes_different_shape(e[i][0]))

#plt.plot(e[0][1], "red",  alpha=1)
#plt.legend(loc="upper left")

norm = mpl.colors.Normalize(vmin=0, vmax=8)
new_cmap = truncate_colormap(cmap, minval=0.0, maxval=0.8, n=100)
plt.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=new_cmap), orientation='horizontal', label='Estimated HW after SB(1)')

plt.show()

In [ ]:
%matplotlib notebook
import matplotlib.pylab as plt
import matplotlib as mpl

plaintext = [0x00 for _ in range(16)]
key = [0x2b, 0x7e, 0x15, 0x16, 0x28, 0xae, 0xd2, 0xa6, 0xab, 0xf7, 0x15, 0x88, 0x09, 0xcf, 0x4f, 0x3c]
set_up(key)
key = expand_key(key)

scope.adc.samples = 9831 - 8523
scope.adc.offset = 8523

cmap = mpl.colormaps['hsv']
print(cmap)
x = np.linspace(0.0, 0.8, 9)

plt.rcParams["figure.figsize"] = (8,4)

plt.figure()

key_int = print_byte_array_in_int(key[0])
print(key_int)

compare = [[] for _ in range(9)]

for t in range(256):
    print(t, end="\r")
    for i in range(16):
        plaintext[i] = (int(key_int[i//4][i%4], 16) ^ t) & 0xFF
    
    plaintext_state = bytes2matrix(plaintext)
    add_round_key(plaintext_state, key[0])
    res_1_0 = matrix2bytes(plaintext_state)[0]
    sub_bytes(plaintext_state)
    res_2_0 = matrix2bytes(plaintext_state)[0]
    state_to_hamming_weight(plaintext_state)
    res = matrix2bytes(plaintext_state)
    
    tmp = encrypt_p_power_repeat_n_abs_bounds_no_average(plaintext, 1000, index_bounds)[0]
    
    # print(f"b_AK = key + {t}. SB(b_AK) = {hex(res_2[0])} with HW {res[0]}. With power trace at index 0 = {tmp[0]}")
    
    #        HW       val before SB, 1st, sum
    compare[res[0]] += [(res_1_0, res_2_0, tmp[0], sum(tmp))]
    
    
    plt.plot(tmp, color=cmap(x[res[0]]), alpha=1, label=f"{i} : HW(SB) = {res[0]}")

norm = mpl.colors.Normalize(vmin=0, vmax=8)
new_cmap = truncate_colormap(cmap, minval=0.0, maxval=0.8, n=100)
plt.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=new_cmap), orientation='horizontal', label='HW after SB(1)')

plt.show()

In [ ]:
print(compare)

In [ ]:
#np.savetxt('output_1_hw0.dat', np.array([np.array(range(9831 - 8523)), np.array(e[0])]).T, fmt='%d %f')
#np.savetxt('output_1_hw4.dat', np.array([np.array(range(9831 - 8523)), np.array(e[1])]).T, fmt='%d %f')
#np.savetxt('output_1_hw8.dat', np.array([np.array(range(9831 - 8523)), np.array(e[2])]).T, fmt='%d %f')

In [ ]:
def sort_pw_trace_16_spikes_different_shape(list_):
    return sum([np.sum(np_array) for np_array in list_])

e = sorted(e, key = sort_pw_trace_16_spikes_different_shape)

#e = np.sort(e.T, axis = 1)

In [ ]:
import math

e_by_hw = []
lower_bound = 0
higher_bound = math.comb(8,0) 
for hw in range(9):
    e_by_hw += [e[:,lower_bound:higher_bound].mean(axis = 1)]
    lower_bound = higher_bound
    higher_bound += math.comb(8,hw+1)
profile_per_spike = (np.array(e_by_hw).T)

In [ ]:
import pickle 

#with open('sb-attack-profile-test-save-full-trace', 'wb') as handle:
#    pickle.dump({"power_trace":e, "index": index_bounds, "key":key, "win":None, "used_fct":"encrypt_p_power_repeat_n_abs_bounds_no_average", "profile_per_spike":None}, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
#with open('sb-attack-profile-test-save-full-trace', 'rb') as handle:
#    dict_ = pickle.load(handle)
#    e = dict_["power_trace"]
#    profile_per_spike = None

#with open('compare-1st-spike-oberse-link-in-trace-same-hw', 'wb') as handle:
#    pickle.dump({"compare":compare}, handle, protocol=pickle.HIGHEST_PROTOCOL)

#with open('compare-1st-spike-oberse-link-in-trace-same-hw', 'rb') as handle:
#    dict_ = pickle.load(handle)
#    compare = dixt_["compare"]


In [ ]:
import math

def index_to_hw(index):
    lower_bound = 0
    higher_bound_exclusive = math.comb(8,0)
    for i in range(9):
        if index >= lower_bound and index < higher_bound_exclusive:
            return i
        lower_bound = higher_bound_exclusive
        higher_bound_exclusive += math.comb(8,i+1) 

def find_HW_from_power_value_list_profile_per_spike_v4(val_list, profile, i):
    closest_index =  np.array([np.sum(np.abs(val_list - profile_elem[i])) for profile_elem in profile]).argmin()
    return index_to_hw(closest_index)

def find_HW_from_power_value_list_list_profile_per_spike_v4(val_list, profile):
    return [find_HW_from_power_value_list_profile_per_spike_v4(val_list[i], profile, i) for i in range(len(list(val_list)))]

def find_HWs_from_pt_profile_per_spike_average_bounds_v4(plaintext, index_bounds=index_bounds, samples=399, profile_per_spike_bounds=e):
    res = np.array([0 for _ in range(16)], dtype=np.float64)
    for i in range(samples):
        e_tmp = encrypt_p_power_repeat_n_abs_bounds_no_average(plaintext, 1, index_bounds)
        res += np.sort(np.array(find_HW_from_power_value_list_list_profile_per_spike_v4(e_tmp, profile_per_spike_bounds)))
    return res / samples

In [ ]:
def find_HW_from_power_value_profile_per_spike(val, profile, i):
    return np.abs(profile[i] - val).argmin()

def find_HWs_from_power_trace_profile_per_spike(val_list, profile):
    return [find_HW_from_power_value_profile_per_spike(val_list[i], profile, i) for i in range(len(list(val_list)))]

def find_HWs_from_pt_profile_per_spike_average(plaintext, index=index_2, samples=399, profile_per_spike=profile_per_spike, window = 3):
    res = np.array([0 for _ in range(16)])
    for i in range(samples):
        e_tmp = encrypt_p_power_repeat_n_abs(plaintext, 1, index, window)
        res += np.sort(np.array(find_HWs_from_power_trace_profile_per_spike(e_tmp, profile_per_spike)))
    return res / samples

# Problem with v2 : not good idea to sort
def find_HW_from_power_value_profile_v2(val, profile, i):
    if val == 0:
        return 0
    argmin_ = np.abs(profile[i] - val).argmin()
    dist = val - profile[i][argmin_]
    if (argmin_ == 0 and dist <= 0) :
        return argmin_ + dist/(profile[i][argmin_ + 1] -  profile[i][argmin_]) 
    if (argmin_ == 8 and dist >= 0):
        return argmin_ + dist/(profile[i][argmin_ - 1] -  profile[i][argmin_]) 
    if dist > 0:
        return argmin_ + dist/(profile[i][argmin_ + 1] -  profile[i][argmin_])  
    else:
        return argmin_ + dist/(profile[i][argmin_] -  profile[i][argmin_ - 1])  

def find_HWs_from_power_trace_profile_v2(val_list, profile):
    for i in range(16):
        for j in range(16):
            val_list[i, j] = find_HW_from_power_value_profile_v2(val_list[i, j], profile, i)
    return np.mean(val_list, axis = 0, where = val_list > 0)

def find_HWs_from_pt_profile_average_v2(plaintext, index=index_2, samples=399, profile_per_spike=profile_per_spike, window = 3):
    res = np.zeros((16, 16))
    counts_ = np.zeros((16, 16))
    for i in range(samples):
        e_tmp = encrypt_p_power_repeat_n_abs(plaintext, 1, index, window)
        range_ = list(range(16))
        argsort_ = e_tmp.argsort()
        for k in range(16):
            res[ argsort_[k], range_[k]] += e_tmp[argsort_[k]]
            counts_[argsort_[k], range_[k]] += 1
    return find_HWs_from_power_trace_profile_v2(np.divide(res, counts_, where = counts_ > 0), profile_per_spike)


def find_HWs_from_power_trace_profile_per_spike_v3(val_list, profile):
    return [find_HW_from_power_value_profile_v2(val_list[i], profile, i) for i in range(len(list(val_list)))]

def find_HWs_from_pt_profile_per_spike_average_v3(plaintext, index=index_2, samples=399, profile_per_spike=profile_per_spike, window = 3):
    res = np.array([0.0 for _ in range(16)], dtype = np.float64)
    for i in range(samples):
        e_tmp = encrypt_p_power_repeat_n_abs(plaintext, 1, index, window)
        res += np.sort(np.array(find_HWs_from_power_trace_profile_per_spike_v3(e_tmp, profile_per_spike)))
    return res / samples

def find_HWs_from_pt_profile_per_spike_average_bounds(plaintext, index_bounds=index_bounds, samples=399, profile_per_spike_bounds=profile_per_spike):
    res = np.array([0 for _ in range(16)], dtype=np.float64)
    for i in range(samples):
        e_tmp = encrypt_p_power_repeat_n_abs_bounds(plaintext, 1, index_bounds)
        res += np.sort(np.array(find_HWs_from_power_trace_profile_per_spike_v3(e_tmp, profile_per_spike_bounds)))
    return res / samples

In [ ]:
plaintext = [0x00 for i in range(16)]

#key = [0x2b, 0x7e, 0x15, 0x16, 0x28, 0xae, 0xd2, 0xa6, 0xab, 0xf7, 0x15, 0x88, 0x09, 0xcf, 0x4f, 0x3c]
key = [i for i in range(16)]

set_up(key)
key = expand_key(key)

scope.adc.samples = 9831 - 8523
scope.adc.offset = 8523

L = []
for t in range(10):
    #plaintext = [t for i in range(16)]
    plaintext[0] = t
    
    plaintext_state = bytes2matrix(plaintext)
    add_round_key(plaintext_state, key[0])
    sub_bytes(plaintext_state)
    state_to_hamming_weight(plaintext_state)
    res = matrix2bytes(plaintext_state)
    res = print_byte_array_in_int(res)
    print(f"For p = [{hex(t)}*16], ordered HW[p_AK] = {np.sort(res)}")

    """
    res = find_HWs_from_pt_profile_average_v2(plaintext, index=index_2, samples=1000, profile_per_spike=profile_per_spike, window = 3)
    print(f"        estimated ordered HW[p_AK] = {res}")

    res = find_HWs_from_pt_profile_per_spike_average(plaintext, index=index_2, samples=1000, profile_per_spike=profile_per_spike, window = 3)
    print(f"        estimated ordered HW[p_AK] = {res}")
    
    res = find_HWs_from_pt_profile_per_spike_average_v3(plaintext, index=index_2, samples=1000, profile_per_spike=profile_per_spike, window = 3)
    print(f"        estimated ordered HW[p_AK] = {res}")
    
    # BEST :
    e_res = find_HWs_from_pt_profile_per_spike_average_bounds(plaintext, index_bounds=index_bounds, samples=1000, profile_per_spike_bounds=profile_per_spike)
    print(f"        estimated ordered HW[p_AK] = {e_res}")
    print(f"#errors = {np.sum(np.abs(np.sort(res) - np.around(e_res)))}")
    """
    
    e_res = find_HWs_from_pt_profile_per_spike_average_bounds_v4(plaintext, index_bounds=index_bounds, samples=1000, profile_per_spike_bounds=e)
    print(f"        estimated ordered HW[p_AK] = {e_res}")
    
    

    L += [e]

In [ ]:
print(profile_per_spike)

In [ ]:
for half_widow_size in range(1, 12):
    for attack in range(4):
        for index in [index_1, index_2]:
            print(f"index : {index}, attack: {attack}, half_widow_size={half_widow_size}")
            plaintext = [0x79, 0x02, 0x04, 0x0b, 0x01, 0x01, 0x04, 0x49, 0xd6, 0xa5, 0x1c, 0x9, 0x13, 0x1, 0x1, 0x0]
            key = [0x2b, 0x7e, 0x15, 0x16, 0x28, 0xae, 0xd2, 0xa6, 0xab, 0xf7, 0x15, 0x88, 0x09, 0xcf, 0x4f, 0x3c]
            set_up(key)
            key = expand_key(key)

            scope.adc.samples = 9831 - 8523
            scope.adc.offset = 8523


            # Find pt s.t HW after SB takes all value 0, ..., 8
            for i in range(16):
                find = False
                t=0
                while find == False:
                    plaintext[i] = t
                    plaintext_state = bytes2matrix(plaintext)
                    add_round_key(plaintext_state, key[0])
                    sub_bytes(plaintext_state)
                    state_to_hamming_weight(plaintext_state)
                    res = matrix2bytes(plaintext_state)
                    res = np.array(print_byte_array_in_int(res))
                    if res[i] == i % 9:
                        find = True
                        # print(f"pt {i} = {hex(t)}")
                    t += 1

            plaintext_state = bytes2matrix(plaintext)
            add_round_key(plaintext_state, key[0])
            sub_bytes(plaintext_state)
            state_to_hamming_weight(plaintext_state)
            res = matrix2bytes(plaintext_state)
            res = np.array(print_byte_array_in_int(res))

            print(f"For p = [{hex(t)}*16], Order HW[p_AK] = {np.sort(res)}")

            if attack == 0:
                e = encrypt_p_power_repeat_n_sort_abs(plaintext, 1000, index, half_widow_size)
            elif attack == 1:
                e = encrypt_p_power_repeat_n_sort(plaintext, 1000, index, half_widow_size)
            elif attack == 2:
                e = encrypt_p_power_repeat_n_sort_max(plaintext, 1000, index, half_widow_size)
            else: 
                e = encrypt_p_power_repeat_n_sort_bounds(plaintext, 1000, index, half_widow_size)
                
                
            print(f"For p = [{hex(t)}*16], e_counts = {e}")
            e = np.array(e)

            if attack in [0, 1, 2]:
                profile = []
                for i in range(9):
                    profile += [e[np.where(np.sort(res) == i)[0]].mean(0)]
                print(profile)

            plaintext = [0x01 for i in range(16)]

            L = []
            for t in range(2):
                plaintext = [t for i in range(16)]

                plaintext_state = bytes2matrix(plaintext)
                add_round_key(plaintext_state, key[0])
                sub_bytes(plaintext_state)
                state_to_hamming_weight(plaintext_state)
                res = matrix2bytes(plaintext_state)
                res = print_byte_array_in_int(res)

                if attack == 0:
                    e = encrypt_p_power_repeat_n_sort_abs(plaintext, 500, index, half_widow_size)
                    print(f"nbr of error of : \n{np.sort(res)}\n{np.array(find_HWs_from_power_trace_profile(e, profile))}\n is : {np.sum(np.abs(np.sort(res) - np.array(find_HWs_from_power_trace_profile(e, profile))))}")
                elif attack == 1:
                    e = encrypt_p_power_repeat_n_sort(plaintext, 500, index, half_widow_size)
                    print(f"nbr of error of : \n{np.sort(res)}\n{np.array(find_HWs_from_power_trace_profile(e, profile))}\n is : {np.sum(np.abs(np.sort(res) - np.array(find_HWs_from_power_trace_profile(e, profile))))}")
                elif attack == 2:
                    e = encrypt_p_power_repeat_n_sort_max(plaintext, 500, index, half_widow_size)
                    print(f"nbr of error of : \n{np.sort(res)}\n{np.array(find_HWs_from_power_trace_profile(e, profile))}\n is : {np.sum(np.abs(np.sort(res) - np.array(find_HWs_from_power_trace_profile(e, profile))))}")
                else: 
                    e = encrypt_p_power_repeat_n_sort_abs(plaintext, 500, index, 5)
                    print(f"nbr of error of : \n{np.sort(res)}\n{e}")
                

In [ ]:
# Build profile
plaintext = [0x79, 0x02, 0x04, 0x0b, 0x01, 0x01, 0x04, 0x49, 0xd6] + [0x00 for _ in range(7)]
key = [0x2b, 0x7e, 0x15, 0x16, 0x28, 0xae, 0xd2, 0xa6, 0xab, 0xf7, 0x15, 0x88, 0x09, 0xcf, 0x4f, 0x3c]
set_up(key)
key = expand_key(key)

scope.adc.samples = 9831 - 8523
scope.adc.offset = 8523

# Find pt s.t HW after SB takes all value 0, ..., 8
for i in range(16):
    find = False
    t=0
    while find == False:
        plaintext[i] = t
        plaintext_state = bytes2matrix(plaintext)
        add_round_key(plaintext_state, key[0])
        sub_bytes(plaintext_state)
        state_to_hamming_weight(plaintext_state)
        res = matrix2bytes(plaintext_state)
        res = np.array(print_byte_array_in_int(res))
        if res[i] == i % 9:
            find = True
            print(f"pt {i} = {hex(t)}")
        t += 1

plaintext_state = bytes2matrix(plaintext)
add_round_key(plaintext_state, key[0])
sub_bytes(plaintext_state)
state_to_hamming_weight(plaintext_state)
res = matrix2bytes(plaintext_state)
res = np.array(print_byte_array_in_int(res))

print(f"For p = [{hex(t)}*16], Order HW[p_AK] = {np.sort(res)}")

#e = encrypt_p_power_repeat_n_sort_abs(plaintext, 5000, index)
e = encrypt_p_power_repeat_n_sort_abs(plaintext, 5000, index_2, 3)
print(f"For p = [{hex(t)}*16], e_counts = {e}")
e = np.array(e)


In [ ]:
profile = []
for i in range(9):
    profile += [e[np.where(np.sort(res) == i)[0]].mean(0)]
print(profile)

In [ ]:
# Build profile v2
key = [0x2b, 0x7e, 0x15, 0x16, 0x28, 0xae, 0xd2, 0xa6, 0xab, 0xf7, 0x15, 0x88, 0x09, 0xcf, 0x4f, 0x3c]
set_up(key)

key = expand_key(key)

scope.adc.samples = 9831 - 8523
scope.adc.offset = 8523

profile = [0x00 for i in range(9)]
profile_count = [0x00 for i in range(9)]
list_ = [[] for i in range(9)]

for t in range(100):
    #t = random.randrange(0, 256)
    print(str(t).zfill(3), end = "\r")
    plaintext = [t for i in range(16)]

    plaintext_state = bytes2matrix(plaintext)
    add_round_key(plaintext_state, key[0])
    sub_bytes(plaintext_state)
    state_to_hamming_weight(plaintext_state)
    res = matrix2bytes(plaintext_state)
    res = np.sort(np.array(print_byte_array_in_int(res)))

    e = encrypt_p_power_repeat_n_sort_abs(plaintext, 100, index)
    # e = encrypt_p_power_repeat_n_sort(plaintext, 100, index)
    
    for i in range(9):
        tmp = e[np.where(res == i)[0]]
        if len(tmp) > 0:
            profile[i] += tmp.mean(0)
            profile_count[i] += 1
            
            list_[i] += list(tmp)
            
        
profile = np.array(profile)/np.array(profile_count)
print(profile)

In [ ]:
# Testing purpose
plaintext = [0x01 for i in range(16)]
key = [0x2b, 0x7e, 0x15, 0x16, 0x28, 0xae, 0xd2, 0xa6, 0xab, 0xf7, 0x15, 0x88, 0x09, 0xcf, 0x4f, 0x3c]
set_up(key)
key = expand_key(key)

scope.adc.samples = 9831 - 8523
scope.adc.offset = 8523

L = []
for t in range(10):
    plaintext = [t for i in range(16)]
    
    plaintext_state = bytes2matrix(plaintext)
    add_round_key(plaintext_state, key[0])
    sub_bytes(plaintext_state)
    state_to_hamming_weight(plaintext_state)
    res = matrix2bytes(plaintext_state)
    res = print_byte_array_in_int(res)
    print(f"For p = [{hex(t)}*16], ordered HW[p_AK] = {np.sort(res)}")

    e = encrypt_p_power_repeat_n_sort(plaintext, 1000, index)
    # e = encrypt_p_power_repeat_n_sort(plaintext, 500, index)
    # print(f"For p = [{hex(t)}*16], e_counts = {e}")
    print(f"        estimated ordered HW[p_AK] = {np.array(find_HWs_from_power_trace_profile(e, profile))}")
    
    L += [e]

## Algo5 : Set hamming weight after SB

In [ ]:
def round_list(list_):
    return [round(elem) for elem in list_]

def extract_diff(base_hw, hw_list):
    res = hw_list.copy()
    for elem in base_hw:
        if elem in res:
            res.remove(elem)
    return res

In [ ]:
plaintext = [0x00 for i in range(16)]

#key = [0x2b, 0x7e, 0x15, 0x16, 0x28, 0xae, 0xd2, 0xa6, 0xab, 0xf7, 0x15, 0x88, 0x09, 0xcf, 0x4f, 0x3c]
key = [i for i in range(16)]

set_up(key)
key = expand_key(key)

scope.adc.samples = 9831 - 8523
scope.adc.offset = 8523

L = []

t = 0x0F
plaintext = [t for i in range(16)]

plaintext_state = bytes2matrix(plaintext)
add_round_key(plaintext_state, key[0])
sub_bytes(plaintext_state)
state_to_hamming_weight(plaintext_state)
res = matrix2bytes(plaintext_state)
res = print_byte_array_in_int(res)
print(f"For p = [{hex(t)}*16], ordered HW[p_AK] = {np.sort(res)}")
e_res = find_HWs_from_pt_profile_per_spike_average_bounds_v4(plaintext, index_bounds=index_bounds, samples=1000, profile_per_spike_bounds=e)
print(f"        estimated ordered HW[p_AK] = {e_res}")

base_hw = round_list(e_res)

# Find s.t p_0 hamming weight after sb(1) is equal to 0
for i in range(256):
    plaintext[0] = i
    
    plaintext_state = bytes2matrix(plaintext)
    add_round_key(plaintext_state, key[0])
    sub_bytes(plaintext_state)
    res = matrix2bytes(plaintext_state)
    if res[0] == 0:
        print(f"Result is {hex(i)}")
    
    e_res = find_HWs_from_pt_profile_per_spike_average_bounds_v4(plaintext, index_bounds=index_bounds, samples=500, profile_per_spike_bounds=e)
    print(extract_diff(base_hw, round_list(e_res)))


### Analysis of comparz

In [ ]:
def compare_rank(compare):
    res = []
    for i in range(len(compare)):
        res += [sorted(compare[i], key= lambda elem: elem[3])]
    return res

def print_compare(compare):
    for i in range(len(compare)):
        for j in range(len(compare[i])):
            print(bin(compare[i][j][0])[2:].zfill(8), bin(compare[i][j][1])[2:].zfill(8), HW[(compare[i][j][0] ^ compare[i][j][1]) & 0xFF], compare[i][j][3])
        print()

In [ ]:
print_compare(compare_rank(compare))

### Disconnect board

In [ ]:
scope.dis()
target.dis()